In [124]:
import pandas as pd
import matplotlib.pyplot as plt

# path
DATA_FOLDER = 'Data/MovieSummaries/'
MOVIE_DATASET = DATA_FOLDER + 'movie.metadata.tsv'

# Dataset loading
movies = pd.read_csv(MOVIE_DATASET, sep='\t', header=None)

# define the name for each columns
movies.columns = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue',
                  'Movie runtime', 'Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)',
                  'Movie genres (Freebase ID:name tuples)']


 ## Prepare the movie dataset


### Removing movies with missing values

In [126]:
# Verify the missing values 
print(movies.isna().sum())

# Drop the movies without release dat, box office revenue or runtime 
movies = movies.dropna(subset=['Movie release date'])

# Drop the box office revenue columns 
movies = movies.drop(columns=['Movie box office revenue'])

if movies.isna().sum().sum() == 0:
    print("No missing values")
else:
    print(movies.isna().sum())


Wikipedia movie ID                               0
Freebase movie ID                                0
Movie name                                       0
Movie release date                            6902
Movie box office revenue                     73340
Movie runtime                                20450
Movie languages (Freebase ID:name tuples)        0
Movie countries (Freebase ID:name tuples)        0
Movie genres (Freebase ID:name tuples)           0
dtype: int64
Wikipedia movie ID                               0
Freebase movie ID                                0
Movie name                                       0
Movie release date                               0
Movie runtime                                16208
Movie languages (Freebase ID:name tuples)        0
Movie countries (Freebase ID:name tuples)        0
Movie genres (Freebase ID:name tuples)           0
dtype: int64


### Create a column 'Release year' which contains the release year of the movie

In [130]:
# Convert the release date column in datetime
#error = coerce convert invalid dates to NaT,
movies['Movie release date'] = pd.to_datetime(movies['Movie release date'], errors='coerce')

# Extract the year and convert it in int
movies['Release year'] = movies['Movie release date'].dt.year
movies = movies.dropna(subset=['Release year'])
movies['Release year'] = movies['Release year'].astype(int)


print(movies[['Movie name', 'Movie release date', 'Release year']].head())

                                          Movie name Movie release date  \
0                                     Ghosts of Mars         2001-08-24   
1  Getting Away with Murder: The JonBenét Ramsey ...         2000-02-16   
2                                        Brun bitter         1988-01-01   
3                                   White Of The Eye         1987-01-01   
4                                  A Woman in Flames         1983-01-01   

   Release year  
0          2001  
1          2000  
2          1988  
3          1987  
4          1983  


 ### Extract and store the movie genres in a column named 'Movie genres'

In [131]:
import json

# Function to extract genres from a JSON-formatted string
def extract_genres(genre_data):
    if isinstance(genre_data, str):
        try:
            # Replace specific occurrences of 'Children's' with 'Children'
            genre_data = genre_data.replace("Children's", "Children")

            # Convert the genre data into a dictionary
            genres_dict = json.loads(genre_data)
            return ", ".join(genres_dict.values())  # Return the genres as a comma-separated string
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {genre_data}")
            return "Unknown_1"  # Return "Unknown" in case of a parsing error
    else:
        return "Unknown_2"  # If the data is not a string, return "Unknown"

# Apply this function to the movie genres column
movies['Movie genres'] = movies['Movie genres (Freebase ID:name tuples)'].apply(extract_genres)

# Check the results
print(movies[['Movie name', 'Movie genres', 'Movie genres (Freebase ID:name tuples)']].head(5))

# Display the count of different genres
print(movies['Movie genres'].value_counts())


                                          Movie name  \
0                                     Ghosts of Mars   
1  Getting Away with Murder: The JonBenét Ramsey ...   
2                                        Brun bitter   
3                                   White Of The Eye   
4                                  A Woman in Flames   

                                        Movie genres  \
0  Thriller, Science Fiction, Horror, Adventure, ...   
1     Mystery, Biographical film, Drama, Crime Drama   
2                               Crime Fiction, Drama   
3  Thriller, Erotic thriller, Psychological thriller   
4                                              Drama   

              Movie genres (Freebase ID:name tuples)  
0  {"/m/01jfsb": "Thriller", "/m/06n90": "Science...  
1  {"/m/02n4kr": "Mystery", "/m/03bxz7": "Biograp...  
2  {"/m/0lsxr": "Crime Fiction", "/m/07s9rl0": "D...  
3  {"/m/01jfsb": "Thriller", "/m/0glj9q": "Erotic...  
4                            {"/m/07s9rl0": "Drama"

### Store the language in a column named 'Movie languages'

In [132]:
import json

# Function to extract the language from a JSON-formatted string
def extract_language(language_data):
    if isinstance(language_data, str):
        try:
            # Convert the language data into a dictionary (assuming it's structured like genres)
            languages_dict = json.loads(language_data)
            # Assuming language is one of the values, you can modify this part depending on the actual structure
            return ", ".join(languages_dict.values())  # Return the language(s) as a comma-separated string
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {language_data}")
            return "Unknown_1"  # Return "Unknown" in case of a parsing error
    else:
        return "Unknown_2"  # If the data is not a string, return "Unknown"

# Apply this function to extract the language
movies['Movie languages'] = movies['Movie languages (Freebase ID:name tuples)'].apply(extract_language)

# Check the results
print(movies[['Movie name', 'Movie languages', 'Movie languages (Freebase ID:name tuples)']].head(5))

# Display the count of different languages
print(movies['Movie languages'].value_counts())


                                          Movie name     Movie languages  \
0                                     Ghosts of Mars    English Language   
1  Getting Away with Murder: The JonBenét Ramsey ...    English Language   
2                                        Brun bitter  Norwegian Language   
3                                   White Of The Eye    English Language   
4                                  A Woman in Flames     German Language   

  Movie languages (Freebase ID:name tuples)  
0        {"/m/02h40lc": "English Language"}  
1        {"/m/02h40lc": "English Language"}  
2        {"/m/05f_3": "Norwegian Language"}  
3        {"/m/02h40lc": "English Language"}  
4         {"/m/04306rv": "German Language"}  
English Language                                                       32677
                                                                       10009
Hindi Language                                                          2799
Spanish Language                    

### Store the movies countries in a column named 'Movie countries'

In [133]:
import json

# Function to extract the countries from a JSON-formatted string
def extract_countries(country_data):
    if isinstance(country_data, str):
        try:
            # Convert the country data into a dictionary (assuming it's structured like genres)
            countries_dict = json.loads(country_data)
            # Assuming countries are one of the values, you can modify this part depending on the actual structure
            return ", ".join(countries_dict.values())  # Return the countries as a comma-separated string
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {country_data}")
            return "Unknown_1"  # Return "Unknown" in case of a parsing error
    else:
        return "Unknown_2"  # If the data is not a string, return "Unknown"

# Apply this function to extract the countries
movies['Movie countries'] = movies['Movie countries (Freebase ID:name tuples)'].apply(extract_countries)

# Check the results
movies[['Movie name', 'Movie countries', 'Movie countries (Freebase ID:name tuples)']].head(5)

,Movie name,Movie countries,Movie countries (Freebase ID:name tuples)
0,Ghosts of Mars,United States of America,"{""/m/09c7w0"": ""United States of America""}"
1,Getting Away with Murder: The JonBenét Ramsey ...,United States of America,"{""/m/09c7w0"": ""United States of America""}"
2,Brun bitter,Norway,"{""/m/05b4w"": ""Norway""}"
3,White Of The Eye,United Kingdom,"{""/m/07ssc"": ""United Kingdom""}"
4,A Woman in Flames,Germany,"{""/m/0345h"": ""Germany""}"


### Fill the missing values of the column 'Movie runtime'

In [ ]:
import requests

def get_run_time_from_wikipedia(wikipedia_id):
    url = f"https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "pageids": wikipedia_id,
        "prop": "extracts",
        "explaintext": True,
        "format": "json"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        page_content = data["query"]["pages"].get(str(wikipedia_id), {}).get("extract", "")
        # Extract the movie runtime from the article
        run_time = extract_run_time(page_content)
        return run_time
    else:
        return None

def extract_run_time(page_content):
    # Extract function
    import re
    match = re.search(r"(\d+)\s*minutes", page_content)
    if match:
        return int(match.group(1))
    return None

# Apply the function to fill the missing values 
movies['Movie runtime'] = movies.apply(
    lambda row: row['Movie runtime'] if pd.notnull(row['Movie runtime']) else get_run_time_from_wikipedia(row['Wikipedia movie ID']),
    axis=1
)


### Add score to the movie dataset

In [53]:
pip install requests textblob

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
# Import necessary libraries
import pandas as pd
import requests
from textblob import TextBlob

API_KEY = '9923aaa2a3b2777bfdeba7f76c97d212'

BASE_URL = 'https://api.themoviedb.org/3/search/movie'

movie_titles = movies[2].tolist()

def get_movie_summary(title):
    params = {
        'api_key': API_KEY,
        'query': title,
        'language': 'en-US'
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['overview']
        else:
            return None
    else:
        print(f"Error fetching data for {title}: {response.status_code}")
        return None

def analyze_sentiment(summary):
    if summary:
        analysis = TextBlob(summary)
        if analysis.sentiment.polarity > 0.5:
            return 5  # Very happy ending
        elif 0.1 < analysis.sentiment.polarity <= 0.5:
            return 4  # Happy ending
        elif -0.1 <= analysis.sentiment.polarity <= 0.1:
            return 3  # Neutral ending
        elif -0.5 < analysis.sentiment.polarity < -0.1:
            return 2  # Sad ending
        else:
            return 1  # Very sad ending
    else:
        return None

num_movies_to_process = 10000
movie_titles_sample = movie_titles[:num_movies_to_process]

results = []
for title in movie_titles_sample:
    summary = get_movie_summary(title)
    score = analyze_sentiment(summary)
    if summary:
        results.append({'Title': title, 'Summary': summary, 'Score': score})

results_df = pd.DataFrame(results)

movie_data_cleaned = movies.copy()
movie_data_cleaned['Score'] = pd.Series(results_df['Score'])

output_file_path = 'movies_dataset_cleaned.tsv'
movie_data_cleaned.to_csv(output_file_path, sep='\t', index=False)

print(f"Cleaned dataset saved to {output_file_path}")